In [1]:

import json

from collections import defaultdict, OrderedDict

# Set your input path here
INPUT_PATH = "./input.txt"  # change this if needed

def parse_json_objects_balanced(text: str):
    objs = []
    i = 0
    n = len(text)
    in_str = False
    escape = False
    depth = 0
    start = None
    while i < n:
        ch = text[i]
        if in_str:
            if escape:
                escape = False
            elif ch == '\\\\':
                escape = True
            elif ch == '"':
                in_str = False
        else:
            if ch == '"':
                in_str = True
            elif ch == '{':
                if depth == 0:
                    start = i
                depth += 1
            elif ch == '}':
                depth -= 1
                if depth == 0 and start is not None:
                    chunk = text[start:i+1]
                    try:
                        objs.append(json.loads(chunk))
                    except Exception:
                        pass
                    start = None
        i += 1
    return objs

with open(INPUT_PATH, "r", encoding="utf-8") as f:
    raw = f.read()

records = parse_json_objects_balanced(raw)
if not records:
    for line in raw.splitlines():
        line = line.strip()
        if not line:
            continue
        try:
            records.append(json.loads(line))
        except Exception:
            continue
if not records:
    raise RuntimeError("Failed to parse any JSON objects from input file.")

by_client = defaultdict(list)
for rec in records:
    rr = rec.get("Results_raw", {})
    cid = rr.get("client_id")
    aidx = rr.get("adapter_idx")
    vloss = rr.get("val_loss")
    if cid is None or aidx is None or vloss is None:
        continue
    by_client[int(cid)].append((int(aidx), float(vloss)))

client_to_adapter = {}
ties = []
for cid, lst in by_client.items():
    lst_sorted = sorted(lst, key=lambda x: (x[1], x[0]))
    min_loss = lst_sorted[0][1]
    tied = [ad for ad, loss in lst_sorted if abs(loss - min_loss) < 1e-12]
    if len(tied) > 1:
        ties.append((cid, min_loss, tied))
    client_to_adapter[cid] = lst_sorted[0][0]

adapter_to_clients = {a: sorted([cid for cid, aidx in client_to_adapter.items() if aidx == a])
                      for a in range(10)}

print("client_to_adapter =")
print(json.dumps(OrderedDict(sorted(client_to_adapter.items())), ensure_ascii=False, indent=2))

print("\\nadapter_to_clients =")
print(json.dumps(adapter_to_clients, ensure_ascii=False, indent=2))

print("\\nnum_ties:", len(ties))
if ties:
    print("ties (cid, min_loss, tied_adapters):")
    print(ties)


client_to_adapter =
{
  "1": 9,
  "2": 9,
  "3": 8,
  "4": 4,
  "5": 1,
  "6": 0,
  "7": 1,
  "8": 0,
  "9": 0,
  "10": 0,
  "11": 0,
  "12": 6,
  "13": 6,
  "14": 0,
  "15": 0,
  "16": 5,
  "17": 5,
  "18": 0,
  "19": 6,
  "20": 6,
  "21": 6,
  "22": 9,
  "23": 4,
  "24": 4,
  "25": 8,
  "26": 5,
  "27": 6,
  "28": 6,
  "29": 2,
  "30": 0,
  "31": 2,
  "32": 5,
  "33": 8,
  "34": 0,
  "35": 0,
  "36": 2,
  "37": 8,
  "38": 0,
  "39": 6,
  "40": 0,
  "41": 8,
  "42": 0,
  "43": 6,
  "44": 8,
  "45": 3,
  "46": 5,
  "47": 8,
  "48": 5,
  "49": 2,
  "50": 5,
  "51": 1,
  "52": 0,
  "53": 8
}
\nadapter_to_clients =
{
  "0": [
    6,
    8,
    9,
    10,
    11,
    14,
    15,
    18,
    30,
    34,
    35,
    38,
    40,
    42,
    52
  ],
  "1": [
    5,
    7,
    51
  ],
  "2": [
    29,
    31,
    36,
    49
  ],
  "3": [
    45
  ],
  "4": [
    4,
    23,
    24
  ],
  "5": [
    16,
    17,
    26,
    32,
    46,
    48,
    50
  ],
  "6": [
    12,
    13,
    19,
    20,
 